---
# --- Day 12: Hot Springs ---
---

In [1]:
from typing import List, Tuple
import re
import numpy as np
from tqdm.notebook import tqdm
import functools

In [2]:
V = lambda *x: np.array(x)

## Load data

In [3]:
full_puzzle_data = True

In [4]:
def read_input(full_puzzle_data: bool) -> List[Tuple[str, tuple]]:
    file_suffix = "" if full_puzzle_data else "_test"
    with open(f"data/day12_input{file_suffix}.txt", "r") as f:
        rows = [row.split() for row in f.read().splitlines()]
    return [(r1, tuple(map(int, r2.split(",")))) for r1, r2 in rows]

In [5]:
data = read_input(full_puzzle_data)

## --- Part One ---

In [6]:
def is_record_valid(record: str, counts: tuple) -> bool:
    formed_groups = re.match(r"^[\.#]+(\.|$)", record)
    if formed_groups is None:
        return True # we don't know yet
    group_string = formed_groups.group()
    group_lengths = [len(g) for g in re.findall(r"[#]+", group_string)]
    if len(group_string) == len(record) and len(group_lengths) != len(counts):
        return False
    return np.array_equal(V(*group_lengths), V(*counts[:len(group_lengths)]))    

In [7]:
@functools.cache
def count_possible_assignments(record: str, counts: tuple) -> bool:
    i = record.find("?")
    if not is_record_valid(record, counts):
        return 0
    elif i == -1:
        return 1
    else:
        r1 = record[:i] + "#" + record[i+1:]
        r2 = record[:i] + "." + record[i+1:]
        return count_possible_assignments(r1, counts) + count_possible_assignments(r2, counts)

In [8]:
sum_counts = 0
for record, counts in tqdm(data, total=len(data)):
    n = count_possible_assignments(record, counts)
    sum_counts += n

  0%|          | 0/1000 [00:00<?, ?it/s]

In [9]:
print(sum_counts)

7195


## --- Part Two ---

In [10]:
@functools.cache
def scan_and_count_solutions(record: str, counts: tuple, num_done_in_group: int=0):
    # Is this a solution? Did we handle and close all groups?
    if len(record) == 0: 
        return int(len(counts) == 0 and num_done_in_group == 0)
    num_solutions = 0
    # If next letter is a "?", we branch
    possible_symbols = [".", "#"] if record[0] == "?" else [record[0]]
    for s in possible_symbols:
        if s == "#":
            # Extend current group
            num_solutions += scan_and_count_solutions(record[1:], counts, num_done_in_group + 1)
        else:
            if num_done_in_group > 0:
                # If we were in a group, check if it can be closed (if not, do not continue)
                if len(counts) > 0 and counts[0] == num_done_in_group:
                    num_solutions += scan_and_count_solutions(record[1:], counts[1:])
            else:
                # If we are not in a group, move on to next symbol
                num_solutions += scan_and_count_solutions(record[1:], counts)
    return num_solutions

In [11]:
sum_counts = 0
for record, counts in tqdm(data, total=len(data)):
    rr = "?".join([record]*5)
    cr = counts*5
    #n = count_possible_assignments(rr, cr)
    n = scan_and_count_solutions(rr + ".", cr)
    sum_counts += n

  0%|          | 0/1000 [00:00<?, ?it/s]

In [12]:
print(sum_counts)

33992866292225
